In [1]:
# =========================
# 📚 AI LESSON PLAN GENERATOR FROM IMAGES
# Google Colab Version
# =========================

!pip install transformers==4.46.2 torch accelerate safetensors gradio python-docx Pillow tqdm -q

# -------------------------
# Step 1: Imports & Setup
# -------------------------
from transformers import pipeline
from docx import Document
import datetime, os, json, gradio as gr

# -------------------------
# Step 2: Robotics Library
# -------------------------
robotics_data = {
    "arduino": {
        "Module": "Arduino Basics and Motor Control",
        "Objectives": [
            "Understand the structure and functioning of Arduino boards",
            "Learn to control LEDs and motors using code"
        ],
        "Activities": [
            "Set up Arduino IDE and connect board",
            "Write and upload LED blink program",
            "Connect a servo motor and control motion via code"
        ]
    },
    "sensor": {
        "Module": "Sensor Integration and Data Reading",
        "Objectives": [
            "Identify types of sensors (IR, ultrasonic, temperature)",
            "Collect and process data from sensors"
        ],
        "Activities": [
            "Connect ultrasonic sensor to Arduino",
            "Read distance data and display on serial monitor"
        ]
    },
    "robot": {
        "Module": "Building a Simple Robot",
        "Objectives": [
            "Understand mechanical and electronic components of a robot",
            "Integrate motion and obstacle detection"
        ],
        "Activities": [
            "Assemble robot chassis",
            "Program forward/backward motion",
            "Implement obstacle avoidance"
        ]
    },
    "ai": {
        "Module": "Introduction to Artificial Intelligence in Robotics",
        "Objectives": [
            "Learn basic AI concepts for robotics",
            "Understand how AI enables decision-making in robots"
        ],
        "Activities": [
            "Use a pre-trained AI model for object detection",
            "Discuss real-world AI applications in robotics"
        ]
    },
    "drone": {
        "Module": "Drone Technology and Control",
        "Objectives": [
            "Understand aerodynamics principles",
            "Learn drone components and flight control basics"
        ],
        "Activities": [
            "Simulate drone flight path using software",
            "Hands-on control with safe indoor demo"
        ]
    }
}

# -------------------------
# Step 3: Model Pipelines
# -------------------------
print("Loading models... (This may take a minute)")
vision_model = pipeline("image-to-text", model="Salesforce/blip-image-captioning-base")
llm = pipeline("text-generation", model="mistralai/Mistral-7B-Instruct-v0.2", device_map="auto")

# -------------------------
# Step 4: Helper Functions
# -------------------------
def suggest_robotics_module(caption):
    caption_lower = caption.lower()
    for keyword, content in robotics_data.items():
        if keyword in caption_lower:
            return content
    return None

def generate_lesson_plan(image_path):
    caption = vision_model(image_path)[0]['generated_text']

    prompt = f"""
    You are a skilled STEM educator.
    Based on this classroom image description: "{caption}",
    create a structured lesson plan including:
    1. Lesson Title
    2. Objectives
    3. Materials Required
    4. Activities (step-by-step)
    5. Assessment Methods
    """

    plan_text = llm(prompt, max_new_tokens=400, temperature=0.7)[0]['generated_text']

    robotics_suggestion = suggest_robotics_module(caption)
    if robotics_suggestion:
        robotics_text = f"\n\n---\n📘 **Suggested Robotics Module:** {robotics_suggestion['Module']}\n\n"
        robotics_text += "**Objectives:**\n"
        for obj in robotics_suggestion['Objectives']:
            robotics_text += f"- {obj}\n"
        robotics_text += "\n**Activities:**\n"
        for act in robotics_suggestion['Activities']:
            robotics_text += f"- {act}\n"
        plan_text += robotics_text

    return caption, plan_text

def save_to_docx(title, plan_text):
    os.makedirs("outputs", exist_ok=True)
    file_path = f"outputs/LessonPlan_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.docx"

    doc = Document()
    doc.add_heading(title, 0)
    doc.add_paragraph(f"Generated on: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M')}\n")

    for line in plan_text.split("\n"):
        if line.strip():
            if any(k in line.lower() for k in ["objective", "material", "activity", "assessment", "module"]):
                doc.add_heading(line.strip(), level=1)
            else:
                doc.add_paragraph(line.strip())
    doc.save(file_path)
    return file_path

# -------------------------
# Step 5: Gradio App
# -------------------------
def lesson_plan_from_image(image):
    caption, plan_text = generate_lesson_plan(image)
    docx_path = save_to_docx("AI-Generated Lesson Plan", plan_text)
    return caption, plan_text, docx_path

demo = gr.Interface(
    fn=lesson_plan_from_image,
    inputs=gr.Image(type="filepath", label="Upload Classroom / STEM Image"),
    outputs=[
        gr.Textbox(label="Scene Description"),
        gr.Textbox(label="Generated Lesson Plan", lines=20),
        gr.File(label="Download Lesson Plan (DOCX)")
    ],
    title="📚 AI Lesson Plan Generator (Robotics-Integrated)",
    description="Upload a classroom or STEM image → get AI-generated lesson plan with objectives, materials, activities, and assessments. Includes Robotics module suggestions."
)

demo.launch(share=True)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 36.7 MB/s eta 0:00:00
Loading models... (This may take a minute)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://96741a47cdea18e290.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
